# Notebook for Testing code


In [7]:
%reset -f
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
import numpy as np
import scipy.io as sio
import sklearn as sk
from sklearn.grid_search import ParameterGrid
from sklearn import decomposition
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import pandas as pd
import datetime
import pickle
import os
from cycling_rnn import *

## Plot settings

In [9]:
cmap = sns.color_palette('RdBu',5)[:2] + sns.color_palette('RdBu',5)[-2:]
sns.set_palette(cmap)
sns.set_context('paper', font_scale=1.5)

## Load stuff

In [71]:
RUN = './saves/8-26/'
param_grid = pickle.load(open(RUN+'npsaves/param_grid.pickle', 'rb'))

sim = 22
cur_params = ParameterGrid(param_grid)[sim]
cur_params

{'activation': 'tanh',
 'beta1': 0.0001,
 'beta2': 0.0001,
 'monkey': 'D',
 'num_neurons': 464,
 'stddev_out': 0.0001,
 'stddev_state': 0.0001}

In [72]:
# List directory
# Number of sims in RUN
total_sims = len(os.listdir(RUN+'tb'))

# load x and y data
x = np.load(RUN+'npsaves/'+str(sim)+'x.npy')
y = np.load(RUN+'npsaves/'+str(sim)+'y.npy')

In [73]:
if cur_params['monkey']=='D':
    data = sio.loadmat('/Users/jeff/Documents/Python/_projects/cyclingRNN/drakeFeb_processed.mat')
else:
    data = sio.loadmat('/Users/jeff/Documents/Python/_projects/cyclingRNN/cousFeb_processed.mat')
    
emg = data['EMG']
m1 = data['M1']

x = x[:emg.shape[0], :emg.shape[1], :]
y = y[:emg.shape[0], :emg.shape[1], :]

## Do stuff

In [74]:
# Iterable Parameters
param_grid = {}
param_grid['activation'] = ['tanh', 'linear']
param_grid['beta1'] = np.logspace(-4, 2, 13)
param_grid['beta2'] = np.logspace(-4, 2, 13)
param_grid['stddev_state'] = np.logspace(-4, 2, 13)
param_grid['stddev_out'] = np.logspace(-4, 2, 13)
param_grid['monkey'] =['D']
param_grid['num_neurons'] = np.concatenate((np.array([100]).astype(int), np.round(np.logspace(1, 3, 13)).astype(int)), axis=0)

val_lengths = [len(v) for k, v in sorted(param_grid.items())]

# Make a copy of param_grid to export to matlab.
param_grid_matlab = copy.deepcopy(param_grid)
param_grid_matlab['val_lengths'] = val_lengths
param_grid_matlab['sorted'] = sorted(param_grid)

# val_lengths ordered by keys
# Important: apparently ParameterGrid() orders lexicographically based on keys.

# Split param_grid to search along hyperparameter axes only
param_grid = parameter_grid_split(param_grid)

In [70]:
ParameterGrid(param_grid)[19]

{'activation': 'tanh',
 'beta1': 0.0001,
 'beta2': 0.0001,
 'monkey': 'D',
 'num_neurons': 100,
 'stddev_out': 0.0001,
 'stddev_state': 0.0001}